In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from sklearn.metrics import mean_squared_error
import time

In [ ]:
# Function to fetch and preprocess historical Bank Nifty data
def preprocess_banknifty_data():
    # Define the end date as today
    end_date = datetime.today()

    # Calculate the start date 6 months ago
    start_date = end_date - timedelta(days=6*30)

    # Fetch Bank Nifty data with a daily interval
    banknifty_data = yf.download('^NSEBANK', start=start_date, end=end_date)

    # Resample data to 5-minute intervals and forward fill missing values
    banknifty_data = banknifty_data.resample('5T').ffill()

    # Extract low, high, open, and close prices
    banknifty_data = banknifty_data[['Low', 'High', 'Open', 'Close']]

    # Rename columns
    banknifty_data.columns = ['Low', 'High', 'Open', 'Close']

    # Give headings to the columns
    banknifty_data.index.name = 'Timestamp'

    return banknifty_data

In [ ]:
# Function to create dataset for LSTM model
def create_dataset(dataset, time_steps=1):
    X, y = [], []
    for i in range(len(dataset) - time_steps - 1):
        a = dataset[i:(i + time_steps), :]
        X.append(a)
        y.append(dataset[i + time_steps, :])
    return np.array(X), np.array(y)

In [ ]:
# Function to train LSTM model
def train_lstm_model(train_data):
    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(train_data)

    # Split data into training and testing sets
    train_size = int(len(scaled_data) * 0.8)
    test_size = len(scaled_data) - train_size
    train_data, test_data = scaled_data[0:train_size, :], scaled_data[train_size:len(scaled_data), :]

    # Define time steps
    time_steps = 20  # Number of time steps to look back

    # Reshape data into X=t and Y=t+1
    X_train, y_train = create_dataset(train_data, time_steps)

    # Define and train the Bi-LSTM model
    model = Sequential([
        Bidirectional(LSTM(units=50, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])),
        LSTM(units=50),
        Dense(units=4)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=100, batch_size=32)

    return model, scaler, time_steps

In [ ]:
# Function to make predictions for the next 20 minutes
def predict_next_20_min_data(model, scaler, time_steps, last_20_min_data):
    last_20_min_data = np.reshape(last_20_min_data, (1, time_steps, last_20_min_data.shape[1]))
    next_20_min_predict = model.predict(last_20_min_data)
    next_20_min_predict = scaler.inverse_transform(next_20_min_predict)
    return next_20_min_predict

In [ ]:
# Function to fetch and display live market data
def show_live_market_data(symbol, model, scaler, time_steps):
    while True:
        # Fetch live data
        live_data = yf.download(symbol, period="1d", interval="1m")

        # Extract last 20 minutes data
        last_20_min_data = live_data[['Low', 'High', 'Open', 'Close']].iloc[-20:]

        # Preprocess last 20 minutes data
        last_20_min_data = scaler.transform(last_20_min_data)

        # Make prediction for the next 20 minutes
        next_20_min_predict = predict_next_20_min_data(model, scaler, time_steps, last_20_min_data)

        # Display live data and prediction
        print("Live Data:")
        print(live_data.tail(1))
        print("Predicted Next 20 Minutes Data:")
        print(next_20_min_predict)

        # Wait for 60 seconds (adjust as needed)
        time.sleep(60)

if __name__ == "__main__":
    # Preprocess historical Bank Nifty data
    banknifty_data = preprocess_banknifty_data()

    # Train Bi-LSTM model on historical data
    model, scaler, time_steps = train_lstm_model(banknifty_data.values)

    # Run the function with the symbol of the stock or index you want to monitor
    show_live_market_data("^NSEBANK", model, scaler, time_steps)


[*********************100%%**********************]  1 of 1 completed


Epoch 1/100


KeyboardInterrupt: 